In [5]:

import xlwings as xw
import os
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [6]:
path = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
nameFile = 'FSC-JAD-01-TEC-61-A1 MV FMECA-Batch 5 LBHD.xlsx'
file_path = os.path.join(path,nameFile)
book = xw.Book(file_path)


In [7]:
sht = book.sheets['5- 4S WBT FMECA']
rng = sht.range('A5:BO27')
df = rng.options(pd.DataFrame, omdex=False, header=True).value

In [9]:
df.head(1)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-CR-001-191_Rev1,4SWBT-LBHD- CR-001,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.5,20.0,10.8,11.5,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.71,0.497037,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.497037,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Less than 6mm remaining,Permanent repair,Scheduled tank inspection,Local pitting on longitudinal bulkhead plating...,Permanent repair


In [ ]:
def func1(row):

    if category == 'Pitting':
        if anomalyType = 'Pit':
            if location = 'Tank boundary plating':
                return ''
